In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F

from netam import framework, models
from netam.common import pick_device, print_parameter_count

In [2]:
# shmoof_data_path = "/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv"
shmoof_data_path = "/Users/matsen/data/shmoof_pcp_2023-11-30_MASKED.csv"
train_df, val_df = framework.load_shmoof_dataframes(shmoof_data_path, val_nickname="51") # , sample_count=5000)

In [3]:
kmer_length = 3
site_count = 500

train_dataset = framework.SHMoofDataset(train_df, kmer_length=kmer_length, site_count=site_count)
val_dataset = framework.SHMoofDataset(val_df, kmer_length=kmer_length, site_count=site_count)

device = pick_device()
train_dataset.to(device)
val_dataset.to(device)

print(f"we have {len(train_dataset)} training examples and {len(val_dataset)} validation examples")

Using Metal Performance Shaders
we have 26592 training examples and 22424 validation examples


In [4]:
# [I 2023-12-03 13:38:46,435] Trial 37 finished with value: 0.0603752824113363 and parameters: {'kernel_size': 19, 'dropout': 0.0, 'embedding_dim': 12, 'filter_count': 12}. Best is trial 37 with value: 0.0603752824113363.

cat_params = {
    "dropout": [0.2, 0.3],
}
int_params = {
    "embedding_dim": (6,7),
    "filter_count": (13,21),
}
float_params = {
}
log_float_params = {
}
# Note that if anything appears below and above, the above gets priority.
fixed_hyperparams = {
    "filter_count": 12,
    "embedding_dim": 12,
    "kernel_size": 9,
    "kmer_length": kmer_length,
    "batch_size": 1024,
    "learning_rate": 0.1,
    "min_learning_rate": 1e-3, # early stopping!
    "dropout": 0.2,
    "l2_regularization_coeff": 1e-6,
    "min_parameter_count": 1000,
    "max_parameter_count": 1525,
}
epochs = 1000
optuna_steps = 50

hyper_burrito = framework.HyperBurrito(pick_device(), train_dataset, val_dataset, models.CNNModel,  epochs=epochs)

hyper_burrito.optuna_optimize(optuna_steps, cat_params, int_params, float_params, log_float_params, fixed_hyperparams)

[I 2023-12-06 16:29:35,227] A new study created in memory with name: no-name-1c43eb87-3d27-4b08-92b5-947ea9eca0de


Using Metal Performance Shaders


Epoch:   5%|▌         | 51/1000 [04:34<1:25:02,  5.38s/it, loss_diff=-1.575e-06, lr=3.2e-5, val_loss=0.06083] 
[I 2023-12-06 16:34:15,122] Trial 0 finished with value: 0.06082492873545761 and parameters: {'dropout': 0.3, 'embedding_dim': 6, 'filter_count': 13}. Best is trial 0 with value: 0.06082492873545761.
Epoch:   4%|▍         | 42/1000 [04:22<1:39:58,  6.26s/it, loss_diff=4.844e-08, lr=3.2e-5, val_loss=0.06074]  
[I 2023-12-06 16:38:44,411] Trial 1 finished with value: 0.060735047165247275 and parameters: {'dropout': 0.3, 'embedding_dim': 7, 'filter_count': 13}. Best is trial 1 with value: 0.060735047165247275.
Epoch:   4%|▍         | 45/1000 [04:23<1:33:04,  5.85s/it, loss_diff=-4.727e-07, lr=3.2e-5, val_loss=0.0607] 
[I 2023-12-06 16:43:13,925] Trial 2 finished with value: 0.060695075975213356 and parameters: {'dropout': 0.3, 'embedding_dim': 7, 'filter_count': 14}. Best is trial 2 with value: 0.060695075975213356.
[I 2023-12-06 16:43:13,931] Trial 3 finished with value: 1000000

Trial rejected. Model has 1561 parameters, not in [1000, 1525]].


Epoch:   5%|▌         | 53/1000 [05:28<1:37:55,  6.20s/it, loss_diff=9.24e-07, lr=3.2e-5, val_loss=0.06074]   
[I 2023-12-06 16:48:48,709] Trial 4 finished with value: 0.06073436275645532 and parameters: {'dropout': 0.2, 'embedding_dim': 7, 'filter_count': 13}. Best is trial 2 with value: 0.060695075975213356.
[I 2023-12-06 16:48:48,714] Trial 5 finished with value: 1000000000.0 and parameters: {'dropout': 0.3, 'embedding_dim': 7, 'filter_count': 17}. Best is trial 2 with value: 0.060695075975213356.


...

Trial rejected. Model has 1567 parameters, not in [1000, 1525]].


Epoch:   5%|▍         | 46/1000 [05:00<1:43:56,  6.54s/it, loss_diff=-1.857e-07, lr=3.2e-5, val_loss=0.06079] 
[I 2023-12-06 20:08:41,280] Trial 47 finished with value: 0.06076427024287111 and parameters: {'dropout': 0.3, 'embedding_dim': 6, 'filter_count': 16}. Best is trial 8 with value: 0.060659909068666265.
[I 2023-12-06 20:08:41,290] Trial 48 finished with value: 1000000000.0 and parameters: {'dropout': 0.3, 'embedding_dim': 7, 'filter_count': 19}. Best is trial 8 with value: 0.060659909068666265.


Trial rejected. Model has 1691 parameters, not in [1000, 1525]].


Epoch:   4%|▍         | 44/1000 [04:52<1:45:54,  6.65s/it, loss_diff=5.531e-08, lr=3.2e-5, val_loss=0.06071]  
[I 2023-12-06 20:13:40,714] Trial 49 finished with value: 0.06070921519559237 and parameters: {'dropout': 0.2, 'embedding_dim': 6, 'filter_count': 20}. Best is trial 8 with value: 0.060659909068666265.


Best Hyperparameters: {'dropout': 0.3, 'embedding_dim': 6, 'filter_count': 20}
Best Validation Loss: 0.060659909068666265
